In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
gwangju_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도_광주시_도서대출정보현황_20180719.csv', encoding='CP949')
yangju_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도_양주시_공공도서관 이용자 도서대출 정보_20200813.csv', encoding='CP949')
uijeongbu_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도_의정부시_공공도서관 도서대출 현황_20200821.csv')
icheon_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도_이천시_공공도서관 도서 대출 현황_20200813.csv', encoding='CP949')
hanam_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도_하남시_공공도서관 대출 현황_20180808.csv', encoding='CP949')
gapyeong_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/경기도가평군_201807공공도서관 대출도서 정보_20190529..csv', encoding='CP949')
guro_book_loan = pd.read_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/raw_data/서울특별시교육청 구로도서관 대출도서현황(반기).csv', encoding='CP949')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
gwangju_book_loan.head(1)

,시군명,도서관명,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일시,데이터기준일자
0,광주시,곤지암도서관,나는 최고의 보청기 전문가다 : 몸으로 승부한 월급쟁이의 도박같은 창업 투쟁기,9.79E+12,325.211,2018-04-01 10:31,2018-07-19


In [5]:
gwangju_book_loan.isnull().sum()

시군명                  0
도서관명                 0
도서명                  0
국제표준도서번호(ISBN)    1804
분류기호(한국십진분류법)        1
대출일시                 0
데이터기준일자              0
dtype: int64

In [6]:
yangju_book_loan.head(1)

,시군명,도서관명,도서명,국제표준도서번호(ISBN),분류기호(ISBN),대출일자,대출시간,관리기관명,데이터기준일자
0,양주시,옥정호수도서관,이상한 과자 가게 전천당. 4,9791164060948,833.8,2020-04-01,10:17,평생교육진흥원,2020-08-13


In [7]:
yangju_book_loan.isnull().sum()

시군명                 0
도서관명                0
도서명                 0
국제표준도서번호(ISBN)    909
분류기호(ISBN)          0
대출일자                0
대출시간                0
관리기관명               0
데이터기준일자             0
dtype: int64

In [8]:
uijeongbu_book_loan.head(1)

,시군명,도서관명,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일자
0,의정부시,가능1동도서관,공자 아저씨네 빵가게,9.79E+12,813.8,2018-04-01


In [9]:
uijeongbu_book_loan.isnull().sum()

시군명                   0
도서관명                  0
도서명                   0
국제표준도서번호(ISBN)     7879
분류기호(한국십진분류법)     13017
대출일자                  0
dtype: int64

In [10]:
icheon_book_loan.head(1)

,소장처,서명,등록번호,분류기호,대출일,대출시간
0,이천시립도서관,너무 재밌어서 잠 못 드는 세계사,EM0224341,909,2019-03-30,19:51:28


In [11]:
icheon_book_loan.isnull().sum()

소장처     0
서명      1
등록번호    1
분류기호    0
대출일     1
대출시간    1
dtype: int64

In [12]:
hanam_book_loan.head(1)

,도서관명,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일시,데이터기준일자
0,신장도서관,최강의 식사 : 인생을 바꾸는 실리콘밸리식 완전무결 2주 다이어트,9.79E+12,594.14,2018-04-01 9:10,2018-08-08


In [13]:
hanam_book_loan.isnull().sum()

도서관명                 0
도서명                  0
국제표준도서번호(ISBN)    1097
분류기호(한국십진분류법)        0
대출일시                 0
데이터기준일자              0
dtype: int64

In [14]:
gapyeong_book_loan.head(1)

,시군명,도서관명,도서명,국제표준도서번호(ISBN),분류기호(한국십진분류법),대출일시,Unnamed: 6,Unnamed: 7
0,가평군,한석봉도서관,톨스토이 단편선,9788971844649,892.83,2018-04-03 22:33,NaN,NaN


In [15]:
gapyeong_book_loan.isnull().sum()

시군명                   0
도서관명                  0
도서명                   0
국제표준도서번호(ISBN)     1562
분류기호(한국십진분류법)         0
대출일시                  0
Unnamed: 6        46729
Unnamed: 7        46729
dtype: int64

In [16]:
guro_book_loan.head(1)

,순번,등록번호,청구기호,서명(부록명),대출일,반납일,반납예정일
0,1,EMG000327806,600.04-ㅌ94ㄹ,리딩 아트 : 293개의 작품으로 만나는 책에 관한 예술,2018-07-01,2018-07-22,2018-07-22


In [17]:
guro_book_loan.isnull().sum()

순번           0
등록번호         0
청구기호       479
서명(부록명)      1
대출일          0
반납일         86
반납예정일        0
dtype: int64

In [18]:
gwangju_book_loan.columns, yangju_book_loan.columns, uijeongbu_book_loan.columns, icheon_book_loan.columns, hanam_book_loan.columns, gapyeong_book_loan.columns, guro_book_loan.columns

(Index(['시군명', '도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(한국십진분류법)', '대출일시',
        '데이터기준일자'],
       dtype='object'),
 Index(['시군명', '도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(ISBN)', '대출일자', '대출시간',
        '관리기관명', '데이터기준일자'],
       dtype='object'),
 Index(['시군명', '도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(한국십진분류법)', '대출일자'], dtype='object'),
 Index(['소장처', '서명', '등록번호', '분류기호', '대출일', '대출시간'], dtype='object'),
 Index(['도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(한국십진분류법)', '대출일시', '데이터기준일자'], dtype='object'),
 Index(['시군명', '도서관명', '도서명', '국제표준도서번호(ISBN)', '분류기호(한국십진분류법)', '대출일시',
        'Unnamed: 6', 'Unnamed: 7'],
       dtype='object'),
 Index(['순번', '등록번호', '청구기호', '서명(부록명)', '대출일', '반납일', '반납예정일'], dtype='object'))

In [19]:
preprocessed_gwangju = gwangju_book_loan.drop(columns=['시군명', '도서관명', '국제표준도서번호(ISBN)', '데이터기준일자'], axis=1)
preprocessed_yangju = yangju_book_loan.drop(['시군명', '도서관명', '국제표준도서번호(ISBN)', '대출시간', '관리기관명', '데이터기준일자'], axis=1)
preprocessed_uijeongbu = uijeongbu_book_loan.drop(['시군명', '도서관명', '국제표준도서번호(ISBN)'], axis=1)
preprocessed_icheon = icheon_book_loan.drop(['소장처', '등록번호', '대출시간'], axis=1)
preprocessed_hanam = hanam_book_loan.drop(['도서관명', '국제표준도서번호(ISBN)', '데이터기준일자'], axis=1)
preprocessed_gapyeong = gapyeong_book_loan.drop(['시군명', '도서관명', '국제표준도서번호(ISBN)', 'Unnamed: 6', 'Unnamed: 7'], axis=1)
preprocessed_guro = guro_book_loan.drop(['순번', '등록번호', '반납일', '반납예정일'], axis=1)

In [20]:
preprocessed_gwangju.head(1)

,도서명,분류기호(한국십진분류법),대출일시
0,나는 최고의 보청기 전문가다 : 몸으로 승부한 월급쟁이의 도박같은 창업 투쟁기,325.211,2018-04-01 10:31


In [21]:
preprocessed_gwangju.dropna(axis=0, inplace=True)
preprocessed_gwangju.isnull().sum()

도서명              0
분류기호(한국십진분류법)    0
대출일시             0
dtype: int64

In [22]:
preprocessed_yangju.head(1)

,도서명,분류기호(ISBN),대출일자
0,이상한 과자 가게 전천당. 4,833.8,2020-04-01


In [23]:
preprocessed_yangju.dropna(axis=0, inplace=True)
preprocessed_yangju.isnull().sum()

도서명           0
분류기호(ISBN)    0
대출일자          0
dtype: int64

In [24]:
preprocessed_uijeongbu.head(1)

,도서명,분류기호(한국십진분류법),대출일자
0,공자 아저씨네 빵가게,813.8,2018-04-01


In [25]:
preprocessed_uijeongbu.dropna(axis=0, inplace=True)
preprocessed_uijeongbu.isnull().sum()

도서명              0
분류기호(한국십진분류법)    0
대출일자             0
dtype: int64

In [26]:
preprocessed_icheon.head(1)

,서명,분류기호,대출일
0,너무 재밌어서 잠 못 드는 세계사,909,2019-03-30


In [27]:
preprocessed_icheon.dropna(axis=0, inplace=True)
preprocessed_icheon.isnull().sum()

서명      0
분류기호    0
대출일     0
dtype: int64

In [28]:
preprocessed_hanam.head(1)

,도서명,분류기호(한국십진분류법),대출일시
0,최강의 식사 : 인생을 바꾸는 실리콘밸리식 완전무결 2주 다이어트,594.14,2018-04-01 9:10


In [29]:
preprocessed_hanam.dropna(axis=0, inplace=True)
preprocessed_hanam.isnull().sum()

도서명              0
분류기호(한국십진분류법)    0
대출일시             0
dtype: int64

In [30]:
preprocessed_gapyeong.head(1)

,도서명,분류기호(한국십진분류법),대출일시
0,톨스토이 단편선,892.83,2018-04-03 22:33


In [31]:
preprocessed_gapyeong.dropna(axis=0, inplace=True)
preprocessed_gapyeong.isnull().sum()

도서명              0
분류기호(한국십진분류법)    0
대출일시             0
dtype: int64

In [32]:
preprocessed_guro.head(1)

,청구기호,서명(부록명),대출일
0,600.04-ㅌ94ㄹ,리딩 아트 : 293개의 작품으로 만나는 책에 관한 예술,2018-07-01


In [33]:
preprocessed_guro.dropna(axis=0, inplace=True)
preprocessed_guro.isnull().sum()

청구기호       0
서명(부록명)    0
대출일        0
dtype: int64

In [34]:
call_num = preprocessed_guro['청구기호'].tolist()

In [36]:
import re

In [37]:
for i, num in enumerate(call_num):
  temp = num.replace(num, num.split('-')[0])
  temp = re.findall('\d+', temp)
  call_num[i] = '.'.join(temp)

In [38]:
preprocessed_guro['청구기호'] = call_num

In [47]:
preprocessed_gwangju.head(1)
preprocessed_gwangju.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_gwangju.csv')

In [48]:
preprocessed_yangju.head(1)
preprocessed_yangju.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_yangju.csv')

In [49]:
preprocessed_uijeongbu.head(1)
preprocessed_uijeongbu.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_uijeongbu.csv')

In [50]:
preprocessed_icheon.head(1)
preprocessed_icheon.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_icheon.csv')

In [51]:
preprocessed_hanam.head(1)
preprocessed_hanam.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_hanam.csv')

In [52]:
preprocessed_gapyeong.head(1)
preprocessed_gapyeong.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_gapyeong.csv')

In [53]:
preprocessed_guro.head(1)
preprocessed_guro.to_csv('/content/drive/My Drive/Project/Library Book Reservation Prediction/preprocessed_data/preprocessed_guro.csv')